In [1]:
import numpy as np
import random
from simulation_helpers import generateAdjacenyMatrix
from optimal_policy import V

In [2]:
n = 100
L = 5
p = 0.02
q = 0
h = 10

In [3]:
round = 10

In [4]:
def selectRandom(Q, L):   
    
    L = min(L, Q.sum())
    
    test = np.zeros(len(Q))
    
    # index of nonzero elements in Q
    index = np.transpose(np.nonzero(Q))    
    index = index.flatten()
   
    # randomly pick L nodes that are still in the graph
    a = np.random.choice(index, L, replace=False)
    
    # generate the test vector
    for i in range(len(a)):
        select = a[i]
        test[select] = 1
        
    
    return test

In [5]:
def highRisk(y, Q, A, removed, L):
    L = min(L, Q.sum())
    
    test = np.zeros(len(Q))
    R = np.zeros(len(Q))
    
    
    R = np.matmul(A, removed)
    removedNeg = 1 - removed
    R[removedNeg == 0] = removedNeg[removedNeg == 0] 

    
    RDup = np.empty_like (R)
    RDup[:] = R
    RDup.sort()
    
    # rank the healthy ones based on the num of removed ones they connected to
    for i in range(1,L+1):
        index = np.where(R == RDup[-i])[0]
        index = np.random.choice(index, 1)
        test[index] = 1
    return test

In [6]:
def removeNodes(y, Q, test, q):
    
    a = np.empty_like (y)
    a[:] = y
    
    # amounce the infected, announce == 0 if that person decides to announce
    announce = np.random.binomial(1,q,len(a))    
    announce = np.multiply(a,announce)  
    announce = 1 - announce
    
    # remove the person who has announced from the graph
    Q[announce == 0] = announce[announce == 0]
    

    # test result according to the test we selected
    testResult = np.multiply(y,test)
    testResult = 1-testResult
    
    # remove the people that are tested infected 
    Q[testResult == 0] = testResult[testResult == 0]
    
    return Q

In [7]:
def infection(y, Q, p, A):
    R = np.zeros(len(Q))
    
    R[Q] = np.matmul(A[Q,:][:,Q], y[Q])

    infection = np.zeros(len(Q))

    for i in range(len(R)):
        source = int(R[i])

        if source > 0:
            v = np.random.binomial(1, p, source)
            if v.sum() > 0:
                infection[i] = 1
    
    return np.clip(y + infection, 0, 1)

In [8]:
def simulationRandom(Q, h, p, q, L, A, n):
    numInf = np.zeros(h)
    
    y = np.zeros(n)
    
    # randomly initialize an infected person
    start = random.randint(0,n-1) 
    y[start] = 1
    
    done = False 
    
    
    for n in range(h): 
            # done if no healthy ppl connected to someone sick or nobody healthy on graph
            R = np.zeros(len(Q))    
            R[Q] = np.matmul(A[Q,:][:,Q], y[Q])            
            done = (max(R) == 0) or (min(y[Q]) == 1)
            
            # else proceed 
            if not done:
                test = selectRandom(Q,L)
                Q = removeNodes(y, Q, test, q)
                y = infection(y, Q, p, A)            
                Q[start] = 0
                
                
            # count the # of infected on the graph
            Q = Q.astype(np.bool)
            inf = y[Q]  
            numInf[n] = int(inf.sum())

        
    return numInf

In [9]:
def simulationRisky(Q, h, p, q, L, A, n):
    
    numInf = np.zeros(h)
    
    y = np.zeros(n)
    start = random.randint(0,n-1) 
    y[start] = 1
    removed = y
    
    for n in range(h):
        # done if no healthy ppl connected to someone sick or nobody healthy on graph
        R = np.zeros(len(Q))    
        R[Q] = np.matmul(A[Q,:][:,Q], y[Q])            
        done = (max(R) == 0) or (min(y[Q]) == 1)
            
        # else proceed 
        if not done:
         
            oldQ = Q.astype(int)
        
            test = highRisk(y, Q, A, removed, L)
        
            Q = removeNodes(y, Q, test, q)
        
            newQ = Q.astype(int)

            removed = oldQ - newQ if n >= 1 else y

        
            y = infection(y, Q, p, A)

        
            if Q[start] == 1:
                Q[start] = 0
        
        # count the # of infected on the graph
        Q = Q.astype(np.bool)
        inf = y[Q]               
        numInf[n] = int(inf.sum())
        
    return numInf

In [10]:
def resultRandom(h, p, q, L, n):
    result1 = np.zeros(h)
    for i in range(round):
        A = generateAdjacenyMatrix(n, 0.8)
        Q = np.ones(n).astype(np.bool)    
        a = simulationRandom(Q, h, p, q, L, A, n)
        result1 += a
        

    return result1 / float(round)

In [11]:
print resultRandom(h, p, q, L, n)

[ 1.   1.8  2.5  3.4  4.6  6.2  9.  11.  15.6 19.1]


In [12]:
def resultRisky(j, p, q, L, n):
    result2 = np.zeros(h)
    for i in range(round):
        A = generateAdjacenyMatrix(n, 0.8)
        Q = np.ones(n).astype(np.bool)
        b = simulationRisky(Q, h, p, q, L, A, n)
        result2 += b
        

    return result2 / float(round)

In [13]:
print resultRisky(h, p, q, L, n)

[ 0.6  0.7  1.2  1.6  2.4  4.   6.6 10.2 13.9 17.4]
